In [ ]:
#import libraries
import pandas as pd
import numpy as np
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import matplotlib.pyplot as plt
import statistics
from datetime import datetime
import seaborn as sns

In [ ]:
#file path
my_path="C:/Users/Annek/Documents/semestre 1/h!ckaton/"

In [ ]:
#data caracteristics
#create a function that determines the dataset caracteristics
def caracteristics (dataset):
    carac=pd.DataFrame(index=dataset.columns.tolist())
  #Types de variables
    carac['types']=dataset.dtypes.tolist()
  #Number of modalities
    modalites=[]
    for i in dataset.columns.tolist():
        modalites.append(len(set(dataset[i].tolist())))
    carac['nombre de modalites']=modalites
  #Number of missing values
    missing_values=[]
    for i in dataset.columns.tolist():
        missing_values.append(dataset[i].isna().sum())
    carac['number of missing values']=missing_values
  #Proportion of missing values
    prop_missing_values=[]
    for i in dataset.columns.tolist():
        prop_missing_values.append(round(((dataset[i].isna().sum()/dataset.shape[0])*100),1))
    carac['Proportion number of missing values(%)']=prop_missing_values
  #Mean
    moyenne=[]
    for i in dataset.columns.tolist():
        if is_numeric_dtype(dataset[i])==True:
            moyenne.append(dataset[i].mean())
        else:
            moyenne.append (np.nan)
    carac['moyenne']=moyenne
      #Maximum
    maxim=[]
    for i in dataset.columns.tolist():
        if is_numeric_dtype(dataset[i])==True:
            maxim.append(dataset[i].max())
        else:
            maxim.append (np.nan)
    carac['maximum']=maxim
      #Minimum
    minim=[]
    for i in dataset.columns.tolist():
        if is_numeric_dtype(dataset[i])==True:
            minim.append(dataset[i].min())
        else:
            minim.append (np.nan)
    carac['minimum']=minim
#Mode
    mode=[]
    for i in dataset.columns.tolist():
        mode.append(statistics.mode(dataset[i].tolist()))
    carac['mode']=mode
    prop_mode=[]
    for i in dataset.columns.tolist():
        if pd.isna(carac.loc[i,"mode"])==False:
            prop_mode.append(round((dataset[dataset[i]==carac.loc[i,"mode"]][i].value_counts().tolist()[0]/dataset[i].value_counts().sum())*100,1))
        else:
            prop_mode.append(round((dataset[i].isna().sum()/(dataset[i].value_counts().sum()+dataset[i].isna().sum()))*100,1))
    carac['proportion mode (%)']=prop_mode

#Afficher les caractéristiques
    return carac

In [ ]:
chunksize = 100_000
chunks = pd.read_csv(my_path+"X_test_Hi5.csv", chunksize=chunksize)
# Example: Process the first chunk
h= None
for chunk in chunks:
    h = chunk
    carac=caracteristics(chunk)#observe characteristics for a chunk
    break  # Stop after processing the first chunk"

In [ ]:
#Columns
list(h.columns)

In [ ]:
#Distribution of target variable
h["piezo_groundwater_level_category"].value_counts()/100000

Dataset seems unbalanced with a minority of low and very low groundwaterlevel

### Let's study "piezo" variables

In [ ]:
carac[carac.index.isin(["piezo_station_investigation_depth",
"piezo_station_altitude",
"piezo_station_longitude",
"piezo_station_latitude",
"piezo_measurement_date",
"piezo_obtention_mode",
"piezo_status",
"piezo_qualification",
"piezo_continuity_code",
"piezo_producer_name",
"piezo_measure_nature_name"
])].sort_values(by=["Proportion number of missing values(%)"],ascending=False)

In [ ]:
#identify stations by coordinates:
h["piezo_station_coordinates"] = h.apply(
    lambda row: f"({row['piezo_station_longitude']}, {row['piezo_station_latitude']})", axis=1
)

In [ ]:
print(len(h["piezo_station_coordinates"].unique()))
print(h[h["piezo_continuity_code"]==1].shape[0])

continuy_code is not a choronicle observable by station as the description mentions...

In [ ]:
print(len(h[h["piezo_continuity_code"]==1]["piezo_measurement_date"].unique()))
print(len(h["piezo_measurement_date"].unique()))

In [ ]:
continuity_code observable by measurement date?

In [ ]:
#Replacing missing values
#let's add some features:
#year
h['piezo_measurement_date_year']=h["piezo_measurement_date"].apply(lambda x:x.year)
#Month
h['piezo_measurement_date_month']=h["piezo_measurement_date"].apply(lambda x:x.month)
#remplace depth by mean of depth by station and observation period
h["piezo_station_investigation_depth"] = h.groupby(["piezo_station_bss_code", "piezo_measurement_date_month"])["piezo_station_investigation_depth"].transform(
    lambda x: x.fillna(x.mean()))


In [ ]:
#let's see
caracs=caracteristics(h)
caracs[caracs.index.isin(["piezo_station_investigation_depth",
"piezo_station_altitude",
"piezo_station_longitude",
"piezo_station_latitude",
"piezo_measurement_date",
"piezo_obtention_mode",
"piezo_status",
"piezo_qualification",
"piezo_continuity_code",
"piezo_producer_name",
"piezo_measure_nature_name","piezo_measurement_date_year","piezo_measurement_date_month"
])].sort_values(by=["Proportion number of missing values(%)"],ascending=False)

Some stations did not observe piezometry for a whole month?

In [3]:
list(h.columns)

['row_index',
 'piezo_station_department_code',
 'piezo_station_update_date',
 'piezo_station_investigation_depth',
 'piezo_station_department_name',
 'piezo_station_commune_code_insee',
 'piezo_station_pe_label',
 'piezo_station_bdlisa_codes',
 'piezo_station_altitude',
 'piezo_station_bss_code',
 'piezo_station_commune_name',
 'piezo_station_longitude',
 'piezo_station_latitude',
 'piezo_station_bss_id',
 'piezo_bss_code',
 'piezo_measurement_date',
 'piezo_obtention_mode',
 'piezo_status',
 'piezo_qualification',
 'piezo_continuity_code',
 'piezo_continuity_name',
 'piezo_producer_code',
 'piezo_producer_name',
 'piezo_measure_nature_code',
 'piezo_measure_nature_name',
 'meteo_id',
 'meteo_name',
 'meteo_latitude',
 'meteo_longitude',
 'meteo_altitude',
 'meteo_date',
 'meteo_rain_height',
 'meteo_DRR',
 'meteo_temperature_min',
 'meteo_time_tn',
 'meteo_temperature_max',
 'meteo_time_tx',
 'meteo_temperature_avg',
 'meteo_temperature_avg_threshold',
 'meteo_temperature_min_ground'